In [1]:
import numpy as np
import pandas as pd

In [2]:
movies_df = pd.read_csv('movies.csv', usecols = ['movieId', 'title'],
                       dtype = {'movieId': 'int32', 'title': 'str'})
rating_df = pd.read_csv('ratings.csv', usecols = ['userId', 'movieId', 'rating'],
                       dtype = {'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [3]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
df = pd.merge(rating_df, movies_df, on = 'movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [6]:
combine_movie_rating = df.dropna(axis = 0)

In [7]:
movie_ratingCount = combine_movie_rating.groupby('title')['rating'].count().reset_index().rename(columns = {'rating': 'totalRatingCount'})[['title', 'totalRatingCount']]
movie_ratingCount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [8]:
print(combine_movie_rating.shape)
print(movie_ratingCount.shape)

(100836, 4)
(9719, 2)


In [9]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount,
                                                         left_on = 'title',
                                                         right_on = 'title',
                                                         how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [10]:
rating_with_totalRatingCount.shape

(100836, 5)

In [11]:
movie_ratingCount.describe()

,totalRatingCount
count,9719.000000
mean,10.375141
std,22.406220
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,329.000000


In [12]:
popularity_threshold = 50
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [13]:
rating_popular_movie.shape

(41362, 5)

In [14]:
movie_features_df = rating_popular_movie.pivot_table(index = 'title', columns = 'userId', values = 'rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [15]:
from scipy.sparse import csr_matrix

In [16]:
# movie_features_df_matrix = csr_matrix(movie_features_df.values)

In [17]:
from sklearn.neighbors import NearestNeighbors

In [20]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [21]:
query_indx = np.random.choice(movie_features_df.shape[0])
print(query_indx)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_indx, :].values.reshape(1, -1), n_neighbors = 6)

327


In [22]:
distances = distances.ravel()
distances

array([0.        , 0.5504297 , 0.5510671 , 0.5549506 , 0.57111007,
       0.5743673 ], dtype=float32)

In [23]:
indices = indices.ravel()
indices

array([327, 401, 205,   8, 238,  13], dtype=int64)

In [24]:
for i in range(len(distances)):
    if i == 0:
        print('Similar movies like {}'.format(movie_features_df.index.values[indices[i]]))
    else:
        print('{0}) {1} with distance {2}'.format(i, movie_features_df.index[indices[i]], distances[i]))

Similar movies like Robin Hood: Men in Tights (1993)
1) Three Musketeers, The (1993) with distance 0.5504297018051147
2) Hot Shots! Part Deux (1993) with distance 0.5510671138763428
3) Ace Ventura: Pet Detective (1994) with distance 0.5549505949020386
4) Last Action Hero (1993) with distance 0.5711100697517395
5) Aladdin (1992) with distance 0.5743672847747803
